In [1]:
%load_ext autoreload
%autoreload 2
%env PYDEVD_DISABLE_FILE_VALIDATION=1

env: PYDEVD_DISABLE_FILE_VALIDATION=1


In [2]:
import sys
sys.path.append('Cpp_lib')
import adjugate_module

In [3]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import math 

from noise_generator import  stack_noise #, interp_stack_noise, interp_dict_noise
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import math 
import os
from tqdm.auto import tqdm
# from robot_dynamics import compute_Y, robot_dynamics, error_dynamics, filter_dynamics
# from matrix_operation import compute_adjugate_old, compute_adjugate_scipy, drem_adapt, regularizing_operator, regressor

# the below solved by solve_ivp
from DREM_routine import robot_dynamics_w_control_adaptation
#  analysis and save utils
from utils import analysis_result, save_params, save_as_pickle
from noise_generator import wrap_interp_func, parse_noise_interp_dict

# import sys
# sys.path.append('~/Dropbox/PJ ModelDetection/DREM/DREM_implementation/Cpp_lib')
# import adjugate_module

In [4]:

##########################
# default parameters
##########################

# Define robot parameters (placeholders)
# theta = np.array([6.3922, 1.4338, 0.0706, 0.0653, 2.4552, 0.2868, 113.6538, 46.1168, 2.0993, 2.6, 2.5, 1.5])
theta = np.array([6.3922, 1.4338, 0.0706, 0.0653, 2.4552, 0.2868, 113.6538, 46.1168, 2.0993, 2.6, 2.5, 1.5])

# initial estimate of theta0
theta_hat0 = 0.0 * theta
pdim = len(theta_hat0)
ndim = 3 # state vector dim
nfilter = 4 # including Y1
q_d = np.array([np.pi/2, -np.pi/2, 0]) # desired position
init_y = np.array([0, 0, np.pi/2, 0, 0, 0])

f_const = {'lambda_phi' : 1.0, 
                'b2'    : 0.2, 
                'b3'    : 0.3, 
                'b4'    : 0.4, 
                'a2'    : 700.0, 
                'a3'    : 700.0, 
                'a4'    : 700.0, 
                'nfilter' : nfilter, 
                'ndim' : ndim, 
                'nrows' : nfilter * ndim, 
                'ncols' : pdim, 
                'theta' : theta,
                'TMAX' : 2, #  seconds
                'SWITCHFLAG' : False,   # if do regime switching
                'THREON' : False,   # if do thresholding for estimated value. Either 1. 'const' constant threshold, 2. 'Sigma_mod' Sigma modification
                'THRERT' : np.array([0.0, 2.0]),  # active when making thresholding on
                'DREMON' : True,        # if use drem adaptation f_theta
                'INTEGRALERROR' : False,        # if use drem adaptation f_theta
                'q_d' : q_d,    # desired position
                'noise_freq' : 5,  # hz
                'sigma_o' : 0.01,
                'init_y' : init_y,
                'theta_hat0' : theta_hat0,
                'ASYMDREM' : False, # update on DREM noise reduction asymptotically in RLS
                }



###########################################
# Initial conditions: joint angles and velocities
tau = np.zeros(ndim)
Lambda = 10 * np.eye(ndim) 
Lambda_i = 5 * np.eye(ndim)  # lambda for integral error compensation
Kv =     10 * np.eye(ndim)
Psi =    10 * np.eye(pdim)
Gamma =  10 * np.eye(pdim)
Sigma_mod =  0.1 * np.eye(pdim)
alpha = 0.50001

control_gains = {
    'Lambda' : Lambda, 
    'Lambda_i' : Lambda_i, 
    'Kv' : Kv,
    'Psi' : Psi,
    'Gamma' : Gamma, 
    'alpha' : alpha,
    'Sigma_mod' : Sigma_mod
}



Main routine

In [11]:

# define experiment name
exp_list = [
            #('00_DREMON_SIGMA000_THREOFF',  True,  0.00,  False     )  ,
            #('01_DREMOFF_SIGMA000_THREOFF', False, 0.00,  False ) ,  
            #('02_DREMON_SIGMA001_THREOFF',  True,  0.001,  False )  ,
            #('03_DREMOFF_SIGMA001_THREOFF', False, 0.001,  False ) ,
            #('04_DREMON_SIGMA002_THREOFF',  True,  0.002,  False ) ,
            #('05_DREMOFF_SIGMA002_THREOFF', False, 0.002,  False ) ,
            #('06_DREMON_SIGMA000_THREON',  True,  0.00,  True  )  ,
            #('07_DREMOFF_SIGMA000_THREON', False, 0.00,  True  ) ,  
            #('08_DREMON_SIGMA001_THREON',  True,  0.01,  True  )  ,
            #('09_DREMOFF_SIGMA001_THREON', False, 0.01,  True  ) ,
            #('10_DREMON_SIGMA010_THREON',  True,  0.05,  True  ) ,
            #('11_DREMOFF_SIGMA010_THREON', False, 0.05,  True  ) ,
            #('13_DREMOFF_SIGMA005_THREOFF', False, 0.005,  False, 100.0, 1.0 ) ,
            #('14_DREMOFF_SIGMA005_THREOFF', True, 0.001,  False, 100.0, 10) ,
            #('12_DREMON_SIGMA005_THREOFF',  True,  0.005,  False ) ,
            #('16_DREMOFF_SIGMA001_THREON', True, 0.001,  True, 100.0, 10) ,
            #('17_DREMOFF_SIGMA001_THREON', False, 0.001,  True, 100.0, 10) ,
            #('18_DREMON_SIGMA001_THREOFF', True, 0.001,  False, 100.0, 10) ,
            #('19_DREMOFF_SIGMA001_THREOFF', False, 0.001,  False, 100.0, 10) ,
            #('27_DREMOFF_SIGMA001_THREON', False, 0.001,  True, 100.0, 10, True) ,
            #('26_DREMOFF_SIGMA001_THREON', True, 0.001,  True, 100.0, 10, True) ,
            #('28_DREMON_SIGMA001_THREOFF', True, 0.001,  False, 100.0, 10, True) ,
            #('29_DREMOFF_SIGMA001_THREOFF', False, 0.001,  False, 100.0, 10, True) ,
            # ('36_DREMON_SIGMA001_THREOFF', True, 0.001,  False, 20.0, 10, True, 0.01) , # condition 19  extension
            #('136_DREMON_SIGMA001_THREOFF', True, 0.001,  False,       100.0, 10, True, 0.01, False, 0.0) , # condition 19  extension
            #('236_DREMON_SIGMA001_THREOFF', True, 0.001,  False,       100.0, 10, True, 0.01, 'RLS', 0.0) , # condition 19  extension
            #('336_DREMON_SIGMA001_THREOFF', True, 0.001,  'Sigma_mod', 100.0, 10, True, 0.01, False, 1.0) , # condition 19  extension
            #('436_DREMON_SIGMA001_THREOFF', True, 0.001,  'Sigma_mod', 100.0, 10, True, 0.01, 'RLS', 1.0) , # condition 19  extension
            #('536_DREMON_SIGMA001_THREOFF', True, 0.001,  'Sigma_mod', 100.0, 10, True, 0.01, False, 0.1) , # condition 19  extension
            #('636_DREMON_SIGMA001_THREOFF', True, 0.001,  'Sigma_mod', 100.0, 10, True, 0.01, 'RLS', 0.1) , # condition 19  extension
            #('736_DREMON_SIGMA001_THREOFF', True, 0.001,  'Sigma_mod', 100.0, 10, True, 0.01, 'RLS_P', 0.1) , # condition 19  extension
            #('836_DREMON_SIGMA001_THREOFF', True, 0.001,  False,       100.0, 10, True, 0.01, 'RLS_P', 0.0) , # condition 19  extension
            ('936_DREMON_SIGMA001_THREOFF', True, 0.001,  False,       1.0, 10, True,  0.01, 'RLS_P', 0.0) , # condition 19  extension
            #('836_DREMON_SIGMA001_THREOFF', True, 0.001,  False,       100.0, 10, False, 0.01, 'RLS_P', 0.0) , # condition 19  extension
              # ('37_DREMOFF_SIGMA001_THREOFF', False, 0.001,  False, 20.0, 10, True, 0.01) , # condition 19  extension
            # ('38_DREMON_SIGMA001_THREON', True, 0.001,  True, 20.0, 10, True, 0.01) , # condition 19  extension
            # ('39_DREMOFF_SIGMA001_THREON', False, 0.001,  True, 20.0, 10, True, 0.01) , # condition 19  extension
            #('28_DREMON_SIGMA001_THREON', True, 0.001,  True, 100.0, 10, True, 0.01) , # condition 19  extension
            #('48_DREMON_SIGMA001_THREON', True, 0.001,  True, 100.0, 10, True, 0.1) , # condition 19  extension
            # ('48_DREMON_SIGMA001_THREON', True, 0.001,  'Sigma_mod', 20.0, 10, True, 0.01) , # condition 19  extension
            # ('58_DREMON_SIGMA001_THREON', True,        0.001,  'Sigma_mod', 20.0, 10, True, 0.01, False) , # condition 19  extension
            #('68_DREMON_SIGMA001_THREON', 'DREM_ONLY', 0.001,  'Sigma_mod', 20.0, 10, True, 0.01, False) , # condition 19  extension
            # ('78_DREMON_SIGMA001_THREON', 'DREM_ONLY', 0.001,  'Sigma_mod', 20.0, 10, True, 0.01, 'RLS') , # condition 19  extension
            ]

for exp_name, DREMON, sig_o, THREON, TMAX, gain_multiple, INTEGRALERROR, Lambda_i, ASYMDREM, gain_Sigma_mod in exp_list:
    
    
    ###########################################
    f_const['TMAX'] = TMAX #  seconds
    # modify default params for experiments
    f_const['DREMON'] = DREMON
    f_const['sigma_o'] = sig_o
    f_const['THREON'] = THREON 
    f_const['INTEGRALERROR'] = INTEGRALERROR
    control_gains['Lambda'] = gain_multiple * np.eye(ndim) 
    control_gains['Lambda_i'] = Lambda_i * np.eye(ndim) 
    control_gains['Kv'] =     gain_multiple * np.eye(ndim)
    control_gains['Psi'] =    gain_multiple * np.eye(pdim)
    control_gains['Gamma'] =  gain_multiple * np.eye(pdim)
    control_gains['Sigma_mod'] =  gain_Sigma_mod * np.eye(pdim)
    f_const['ASYMDREM'] = ASYMDREM
    
    ###########################################
    
    # define filtered regressors
    Yphi =   np.zeros((f_const['nfilter'] * f_const['ndim'], f_const['ncols']))
    tauphi = np.zeros((f_const['nrows']))
    ei0 = np.zeros((f_const['ndim']))
    phi2sum0 = [0] # integration of phi2
    P0 = np.zeros((f_const['nfilter'] * f_const['ndim'], f_const['ncols'])) # integration of Ya.T@Ya 

     # Time points to evaluate the solution
    t = np.linspace(0, f_const['TMAX'], int(f_const['TMAX']*f_const['noise_freq'] + 1))
     # Time span for the simulation
    t_span = [0.0, f_const['TMAX']]  
    
    # noise
    np.random.seed(seed=32)
    noise_info =  stack_noise(f_const['ndim'], n_samples=int(f_const['TMAX']*f_const['noise_freq'] + 1), tbegin=0, tend=f_const['TMAX'], dbgfig=0, sigma_o=f_const['sigma_o'])
    noise_interp_dict = wrap_interp_func(t, noise_info) # wrapper for interpolate computation
    
    # initial condition
    ytheta0 = np.concatenate([f_const['init_y'], f_const['theta_hat0'], Yphi.flatten(), tauphi.flatten(), ei0.flatten(), phi2sum0, P0.flatten()])  # (q, q_dot, theta_hat, Yf, tauf)
    
   
   
    # prepare noise interp function
 
    # Initialize the progress bar
    initial_time, final_time = t_span  # Assuming t_span is defined somewhere as [initial_time, final_time]
    total_time = final_time - initial_time
    pbar = tqdm(total=total_time)
    # Create a tqdm progress bar
    
    # Wrapper function to update progress bar based on current time t
    def wrapper(t, y, *args):
        current_progress = t - initial_time
        # Update the progress bar to reflect current progress
        # Note: set_postfix can be used to display additional information
        pbar.n = current_progress
        pbar.last_print_n = current_progress
        pbar.refresh()
        return robot_dynamics_w_control_adaptation(t, y, *args)


    # delete prior Solve 
    # try:
    #     del sol, y  # Attempt to delete 'my_variable'
    # except NameError:
    #     print("sol, y is not defined or already deleted.")
 
    # Evaluate the solution
    #sol = solve_ivp(robot_dynamics_w_control_adaptation, t_span, ytheta0, args=( control_gains, f_const, noise_info, ), dense_output=True, rtol=1e-3, atol=1e-6, events=callback)
    #sol = solve_ivp(wrapper, t_span, ytheta0, args=( control_gains, f_const, noise_interp_dict, ), dense_output=False, rtol=1, atol=1, max_step=0.0001, t_eval=t)
    sol = solve_ivp(wrapper, t_span, ytheta0, args=( control_gains, f_const, noise_interp_dict, ), dense_output=False, rtol=1e-3, atol=1e-4, t_eval=t)
    y = sol.y
    #sol = solve_ivp(wrapper, t_span, ytheta0, args=( control_gains, f_const, noise_interp_dict, ), dense_output=True, rtol=1e-3, atol=1e-6, t_eval=t)
    #y = sol.sol(t)
    
    
    # Close the progress bar after the integration is complete
    pbar.close()


    ####################################################    
    # post process 
    ####################################################    

    # show analysis resutls
    save_dir = 'img/'
    savepath = os.path.join(save_dir, exp_name)
    analysis_result(t,y,theta_hat0, f_const, save=True, savepath=savepath, figshow=0, )
    
    # merge dictionaries
    parameters = f_const | control_gains | noise_info
    save_params(parameters,   savepath=savepath)
    save_params(f_const,      savepath=savepath, filename='f_const.json')
    save_params(control_gains,savepath=savepath, filename='control_gains.json')
    save_params(noise_info,   savepath=savepath, filename='noise_info.json')

    # save result
    save_as_pickle(sol, savepath + '/solution.pickle')



  0%|          | 0/100.0 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [160]:
sol.y[-2]

NameError: name 'sol' is not defined

In [161]:
import pickle
file_exp16 = '/Users/hiroki/Dropbox/PJ ModelDetection/DREM/DREM_implementation/img/936_DREMON_SIGMA001_THREOFF/solution.pickle'
with open(file_exp16, 'rb') as f:
    foo = pickle.load(f)

In [197]:
foo.y[:,-1][-144:].reshape(12,12)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [37]:

with open(file_exp16, 'wb') as f:
    pickle.dump(foo, f)

In [36]:
foo.sol = None

In [16]:

# Function to display memory usage of variables
def display_memory_usage():
    print("Variable Name\tMemory Usage")
    for variable_name in globals().copy():
        # Filter out system and module variables
        if not variable_name.startswith("_") and variable_name not in sys.modules:
            print(f"{variable_name}\t{sys.getsizeof(globals()[variable_name])} bytes")

# Call the function to display memory usage
display_memory_usage()

Variable Name	Memory Usage
In	248 bytes
Out	224 bytes
get_ipython	64 bytes
exit	56 bytes
quit	56 bytes
open	152 bytes
np	72 bytes
solve_ivp	152 bytes
plt	72 bytes
stack_noise	152 bytes
robot_dynamics_w_control_adaptation	152 bytes
analysis_result	152 bytes
save_params	152 bytes
save_as_pickle	152 bytes
wrap_interp_func	152 bytes
parse_noise_interp_dict	152 bytes
theta	208 bytes
theta_hat0	208 bytes
pdim	28 bytes
ndim	28 bytes
nfilter	28 bytes
q_d	136 bytes
init_y	160 bytes
f_const	832 bytes
tau	136 bytes
Lambda	200 bytes
Kv	200 bytes
Psi	1280 bytes
Gamma	1280 bytes
alpha	24 bytes
control_gains	184 bytes
exp_list	72 bytes
exp_name	75 bytes
DREMON	28 bytes
sig_o	24 bytes
THREON	28 bytes
TMAX	24 bytes
gain_multiple	28 bytes
Yphi	1280 bytes
tauphi	208 bytes
t	112 bytes
t_span	72 bytes
noise_info	184 bytes
noise_interp_dict	184 bytes
ytheta0	1504 bytes
initial_time	24 bytes
final_time	24 bytes
total_time	24 bytes
pbar	56 bytes
wrapper	152 bytes
save_dir	53 bytes
savepath	79 bytes
parameters

In [18]:
sol

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  2.000e-01 ...  9.980e+01  1.000e+02]
        y: [[ 0.000e+00  1.328e+00 ...  1.571e+00  1.570e+00]
            [ 0.000e+00 -1.373e+00 ... -1.570e+00 -1.570e+00]
            ...
            [ 0.000e+00  2.026e+03 ...  2.107e+00  5.226e-01]
            [ 0.000e+00 -2.647e+02 ...  2.318e-01 -1.471e-01]]
      sol: None
 t_events: None
 y_events: None
     nfev: 1007030
     njev: 0
      nlu: 0

In [ ]:
f_const['noise_freq']

10

In [ ]:
# def theta_dot_thre(theta_hat, theta, ratio, theta_hat_dot):
#     # ratio indicates parameter estimation errror allowance
#     # currently only define for theta>0 case
# 
#     def are_all_positive(numbers):
#         return all(n > 0 for n in numbers)
#     
#     # Example usage
# 
# 
#     if are_all_positive(theta):
#         theta_hat_dot [( theta * ratio[0] > theta_hat) & (theta_hat_dot<0) ] = 0
#         theta_hat_dot [( theta * ratio[1] < theta_hat) & (theta_hat_dot>0) ] = 0
#         return theta_hat_dot
# 
#     else:
#         raise ValueError("Not all elements are positive.")
# 
# 
# 
# numbers = np.array([1,2,3,4,5])
# estimate = np.random.rand(5) - 1/2
# theta_hat_dot = np.random.rand(5) - 1/2
# theta_dot_thre(estimate, numbers, np.array([0.2, 2]), theta_hat_dot)

In [15]:
    y = sol.y
    
    # Close the progress bar after the integration is complete
    pbar.close()


    ####################################################    
    # post process 
    ####################################################    

    # show analysis resutls
    save_dir = 'img/'
    savepath = os.path.join(save_dir, exp_name)
    analysis_result(t,y,theta_hat0, f_const, save=True, savepath=savepath, figshow=0, )
    
    # merge dictionaries
    parameters = f_const | control_gains | noise_info
    save_params(parameters,   savepath=savepath)
    save_params(f_const,      savepath=savepath, filename='f_const.json')
    save_params(control_gains,savepath=savepath, filename='control_gains.json')
    save_params(noise_info,   savepath=savepath, filename='noise_info.json')

    # save result
    save_as_pickle(sol, savepath + '/solution.pickle')


Directory 'img/18_DREMON_SIGMA001_THREOFF' was created.
Parameters have been saved to img/18_DREMON_SIGMA001_THREOFF/parameters.json.
Parameters have been saved to img/18_DREMON_SIGMA001_THREOFF/f_const.json.
Parameters have been saved to img/18_DREMON_SIGMA001_THREOFF/control_gains.json.
Parameters have been saved to img/18_DREMON_SIGMA001_THREOFF/noise_info.json.
Variable successfully saved to img/18_DREMON_SIGMA001_THREOFF/solution.pickle


In [ ]:
import DREM_routine
plt.hist(DREM_routine.elapsed_dict['et1'], bins=100)
plt.show()
plt.hist(DREM_routine.elapsed_dict['et2'], bins=100)
plt.show()
plt.hist(DREM_routine.elapsed_dict['et3'], bins=100)
plt.show()
plt.hist(DREM_routine.elapsed_dict['et4'], bins=100)
plt.show()
plt.hist(DREM_routine.elapsed_dict['et5'], bins=100)
plt.show()